In [25]:
import speech_recognition as sr
from gtts import gTTS
import playsound
import openai
import time
from pydub import AudioSegment
from pydub.playback import play
import os
import requests
import json
from IPython.display import Audio
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import cv2

In [26]:
# Camera Part
def using_camera():
    now = time
    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)

    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    image_name = now.strftime('%Y-%m-%d_%H:%M:%S') + 'jpg'
    cv2.imwrite(image_name, frame)

    cap_release()
    cv2.destroyAllWindows()
    return image_name

In [15]:
# Model Part
categories = ["가스레인지", "다리미", "소화기", "커피포트"] # 대회 시간에 맞추기 위해 최소한의 카테고리로 구성하였습니다.
model = keras.models.load_model('D:/영웅/Documents/영웅/대회/2023년 SW미래채움 고등부/문서/심화 프로젝트/CNN_Classification.h5', compile=False) # 학습 후 저장 한 모델 불러오기
def using_model(input_data):
  img = Image.open(input_data)
  img = img.convert("RGB")
  img = img.resize((64, 64))
  data = np.asarray(img)
  X = np.array(data)
  X = X.astype("float") / 256
  X = X.reshape(-1, 64, 64, 3)

  pred = model.predict(X)
  result = [np.argmax(value) for value in pred]
  return categories[result[0]]

In [17]:
# TTS Part
def speak(text):
    now = time
    tts = gTTS(text=text, lang='ko')
    filename = now.strftime('%Y-%m-%d_%H:%M:%S') + '.mp3'
    tts.save(filename)
    audio = Audio(filename)
    display.display(audio)

In [27]:
r = sr.Recognizer()

In [28]:
# Recording Part
def Recording():
    with sr.Microphone() as source:
        print('녹음 시작')
        audio = r.listen(source)
        print('녹음 끝')

    try:
        text = r.recognize_google(audio, language='ko')

    except sr.UnknownValueError:
        print('인식 실패')

    except sr.RequestError as e:
        print(f"요청 실패 : {e}")
    return text

In [29]:
# Translator Part
def translator(text):
  client_id = ""
  client_secret = ""
  data = {'text' : text,
          'source' : 'ko',
          'target' : 'en'}
  url = "https://openapi.naver.com/v1/papago/n2mt"
  header = {"X-Naver-Client-id":client_id,
            "X-Naver-client-Secret":client_secret}
  response = requests.post(url, headers=header, data=data)
  rescode = response.status_code
  if(rescode==200):
    send_data = response.json()
    trans_data = (send_data['message']['result']['translatedText'])
    return trans_data
  else:
    print("Error Code:", rescode)

In [ ]:
# GPT Part
open.api_key = ""

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import IPython.display as display

def display_image(url) :
  image_response = requests.get(url)
  img = Image.open(BytesIO(image_response.content))

  display.display(img)

In [ ]:
def using_gpt(input_prompt):
  text_response = openai.ChatCompletion.create(
      model="ft:gpt-3.5-turbo-0613:swhub-ai:chat-giseong07:86GUjiXW",
      max_tokens=3000,
      messages=[
          {"role": "system", "content": "시각 장애인에 대해 편견이 없고 시각 장애인이 이해하기 쉽도록 질문에 대답해 주시고 친절하게 설명해 주시는 사회복지사"},
          {"role": "user", "content": input_prompt}])
  return text_response["choices"][0].message.content

In [ ]:
speak('질문과 그림 생성 중 한가지를 골라 이야기 해주세요. ') # TTS
select_input = Recording() # Recording
print(f"당신의 답변 : {select_input}")

# 질문이라고 이야기 하신 경우 gpt에게 음성으로 프롬프트에 들어갈 말을 해주세요.
if select_input in '질문':
  # 질문 받을 프롬프트를 이미지 또는 음성 둘 중 선택할 수 있습니다.
  speak('프롬프트를 음성으로 입력할지 사진 촬영으로 입력할지 선택하여 이야기 해주세요.') # speak
  selection_mode = Recording() # Recording
  print(f"당신의 답변 : {selection_mode}")
  if selection_mode  in '사진 촬영':
    speak('사진을 촬영해 주세요.') # speak
    image_name = using_camera()
    input_prompt = using_model('D:/영웅/Documents/영웅/대회/2023년 SW미래채움 고등부/문서/심화 프로젝트/') + image_name # using_model
    print('Model 예측 완료')
    input_prompt = input_prompt + ' 사용법을 알려줘'
    time.sleep(5)
    speak('텍스트와 TTS를 생성하고 있습니다. 잠시만 기다려주세요.') # Speak
    output_text = using_gpt(input_prompt) # using_gpt
    print(output_text) # text 출력
    speak(output_text) # speak
  else:
    speak('프롬프트에 들어갈 문장을 이야기 해주세요.') # TTS
    input_prompt = Recording() # Recording
    print(f"당신의 답변 : {input_prompt}")
    speak('텍스트와 TTS를 생성하고 있습니다. 잠시만 기다려주세요.') # Speak
    output_text = using_gpt(input_prompt) # using_gpt
    print(output_text) # text 출력
    speak(output_text) # speak
    # 그림 생성이라고 이야기 하신 경우에 프롬프트에 그리고 싶은 것을 말해주시고 프롬프트는 영어로 번역되어 들어갑니다.
elif select_input in '그림 생성':
  speak('프롬프트에 들어갈 문장을 이야기 해주세요.') # TTS
  input_prompt = Recording() # Recording
  print(f"당신의 답변 : {input_prompt}")
  image_response = openai.Image.create(
    prompt=translator(input_prompt), # Translate
    n=1,
    size="512x512" # 256X256
  )
  display_image(image_response["data"][0]['url']) # Show Image
